In [ ]:
# Fractional Delay Line

from scipy import signal
from scipy.fftpack import fft, ifft
import numpy as np

from IPython.display import Audio 
import soundfile as sf


import numpy as np

x, Fs = sf.read("snare.wav")

import numpy as np

allpass_params = [
    (210, 0.75),   # APF1
    (158, 0.75),   # APF2
    (561, 0.625),  # APF3
    (410, 0.625)   # APF4
]

predelay = 30000         # e.g. 500 samples (~11 ms @ 44.1kHz)
lpf_a = 0.9            # darkish initial LPF


class DelayLine:
    def __init__(self, max_size):
        """
        A simple circular buffer delay line.
        max_size should be >= the largest delay you plan to use.
        """
        self.size = max_size
        self.buffer = np.zeros(max_size, dtype=np.float32)
        self.write_pos = 0

    def write(self, sample):
        """
        Write a single sample into the delay line.
        Advance the write pointer (wrapping with modulo).
        """
        self.buffer[self.write_pos] = sample
        self.write_pos = (self.write_pos + 1) % self.size

    def tap(self, delay_samples):
        """
        Read a sample 'delay_samples' behind the write pointer.
        """
        read_index = (self.write_pos - delay_samples) % self.size
        return self.buffer[read_index]

class OnePoleLPF:
    def __init__(self, a=0.5):
        """
        a in [0,1), typical range ~0.2-0.95
        The closer to 1, the darker the sound.
        """
        self.a = a
        self.y_prev = 0.0  # filter memory

    def process_sample(self, x):
        """
        Process one sample 'x' through the 1-pole low-pass filter.
        """
        y = self.a * self.y_prev + (1.0 - self.a) * x
        self.y_prev = y
        return y

class AllPassFilter:
    def __init__(self, delay_samples, g, max_size):
        """
        All-pass filter using a circular buffer for the delay.
        delay_samples: D
        g: feedback/feedforward coefficient
        max_size: must be >= delay_samples
        """
        self.D = delay_samples
        self.g = g
        self.delay_line = DelayLine(max_size)

    def process_sample(self, x: float) -> float:
        """
        One typical all-pass implementation:
          y = delayed_value - g * x
          new_value = x + g * y
          delay_line.write(new_value)

        Then output y.

        Alternatively, the sign might be reversed:
          y = -g*x + delayed_value
          new_value = x + g*y
        They are mathematically similar, just a matter of sign convention.
        """
        delayed_value = self.delay_line.tap(self.D)
        y = delayed_value - self.g * x
        new_value = x + self.g * y
        self.delay_line.write(new_value)
        return y

def apply_early_reflections(
    x: np.ndarray,
    predelay_samples: int,
    lpf_a: float,
    ap_params: list,
    max_size: int = 20000
):
    """
    :param x:         Input audio (1D numpy array).
    :param predelay_samples: Delay in samples for the predelay.
    :param lpf_a:     The 1-pole LPF coefficient 'a' for the initial filter.
    :param ap_params: List of (D, g) tuples for each allpass filter in series.
    :param max_size:  For each internal DelayLine, must be >= max needed delay.

    Returns a numpy array with enough extra length so that
    the entire "chain" output is captured.
    """

    # 1) Setup our submodules:
    # -- A simple DelayLine for predelay
    predelay_line = DelayLine(max_size=max_size)
    # -- The 1-pole LPF
    lpf = OnePoleLPF(a=lpf_a)
    # -- The series of AllPassFilters
    allpasses = []
    for (D, g) in ap_params:
        allpasses.append(AllPassFilter(D, g, max_size=max_size))

    # 2) Determine how long the output should be
    sum_of_ap_delays = sum(D for (D, _) in ap_params)
    out_len = len(x) + predelay_samples + sum_of_ap_delays

    y = np.zeros(out_len, dtype=x.dtype)

    # 3) Process sample-by-sample
    for n in range(out_len):
        # If within input range, feed x[n], else 0.0
        in_sample = x[n] if n < len(x) else 0.0

        # A) Predelay
        predelay_line.write(in_sample)
        delayed_sample = predelay_line.tap(predelay_samples)

        # B) LPF
        filtered_sample = lpf.process_sample(delayed_sample)

        # C) Series Allpass Filters
        ap_out = filtered_sample
        for apf in allpasses:
            ap_out = apf.process_sample(ap_out)

        # D) Final output is after the last AP
        y[n] = ap_out

    return y


earlyRefProcessed = apply_early_reflections(
    x,
    predelay_samples=predelay,
    lpf_a=lpf_a,
    ap_params=allpass_params,
    max_size=50000  # must be >= largest AP delay (561 or 410)
)

class ModulatedAllPass:
    def __init__(self, max_size: int, g: float, initial_delay: float):
        """
        A modulated Schroeder allpass filter with fractional delay.
        
        Args:
            max_size:       Size of internal buffer (must be >= the max delay you need).
            g:              Allpass feedback/feedforward coefficient.
            initial_delay:  Initial delay in samples (can be float).
        """
        self.size = max_size
        self.buffer = np.zeros(max_size, dtype=np.float32)
        self.write_pos = 0

        self.g = g
        self.delay = initial_delay  # in samples, can be float

    def set_delay(self, new_delay: float):
        """
        Update the internal delay time (in samples). 
        This can be changed each sample if you want continuous modulation.
        """
        self.delay = new_delay

    def process_sample(self, x: float) -> float:
        """
        Process one input sample through the modulated allpass filter.

        Allpass formula (one variant):
          delayed_val = fractionalTap(delay)
          y = delayed_val - g * x
          new_val = x + g * y
          write(new_val)
          output = y
        """
        delayed_val = self._fractional_tap(self.delay)
        y = delayed_val - self.g * x
        new_val = x + self.g * y

        # Write new_val into buffer
        self.buffer[self.write_pos] = new_val
        self.write_pos = (self.write_pos + 1) % self.size

        return y

    def tap(self, tap_delay: float) -> float:
        """
        If you want to 'grab' a signal from inside the delay line 
        (for creating taps or crossfeeds), you can call this.
        """
        return self._fractional_tap(tap_delay)

    def _fractional_tap(self, delay_samples: float) -> float:
        """
        Reads from the circular buffer 'delay_samples' behind write_pos,
        using linear interpolation for fractional positions.
        """
        # Which sample index are we reading from?
        # We'll break delay_samples into integer + fractional parts:
        d_int = int(np.floor(delay_samples))
        frac = delay_samples - d_int

        # 'base_index' is the integer index behind write_pos
        base_index = self.write_pos - d_int

        # We want to read at base_index and base_index-1 to interpolate
        idx_a = base_index % self.size
        idx_b = (base_index - 1) % self.size

        # Sample values
        val_a = self.buffer[idx_a]
        val_b = self.buffer[idx_b]

        # Linear interpolate:  val = val_a + frac*(val_b - val_a)
        val = val_a + frac * (val_b - val_a)
        return val


# Listen in a Jupyter notebook:
Audio(earlyRefProcessed, rate=Fs)
